# Import Data

In [1]:
PROJECT_PATH = "/Users/donghur/Desktop/Research/Murthy/BM_GPU"
PROJECT_PATH_BMGPU = "/Users/donghur/Desktop/Research/Murthy/BM_GPU"

In [2]:
import sys, os, random, matplotlib, pickle, yaml, hdbscan
import numpy as np
from sklearn.utils import shuffle
import seaborn as sns
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from glob2 import glob

sys.path.append(PROJECT_PATH)
sys.path.append(PROJECT_PATH_BMGPU)

from utils.data import Dataset
from utils.figure import plot_HDBSCAN
from cluster import HDBSCAN
%load_ext autoreload
%autoreload 1
%aimport helper
%aimport utils.data
%aimport utils.figure

In [3]:
config_path = f"{PROJECT_PATH_BMGPU}/config_ant.yaml"
Data = Dataset(PROJECT_PATH, config_path)
Data.load_data()

# configuration
INFO = Data.info
INFO_values = Data.info_values
config = Data.config
skeleton = config['skeleton']
skeleton_color= config['skeleton_color']

# features
bp = Data.data_obj['bodypoints']
rot_bp = Data.data_obj['rotated_bodypoints']
angles = Data.data_obj['angles']
limbs = Data.data_obj['limbs']
angle_power = Data.data_obj['angle_power']
limb_power = Data.data_obj['limb_power']

# embeddings
all_embed = Data.data_obj['all_embeddings']
all_postural_embed = Data.data_obj['all_postural_embeddings']
all_kinematic_embed = Data.data_obj['all_kinematic_embeddings']
angle_kinematic_embed = Data.data_obj['angle_kinematic_embeddings']

Loading config.yaml ...
Finished loading config
Loading INFO.yaml ...


  1%|▏         | 2/141 [00:00<00:07, 17.71it/s]

Finished loading INFO


100%|██████████| 141/141 [00:31<00:00,  4.52it/s]


# Find Undefined Frames

In [4]:
bp_nan_fr = np.unique( np.where(np.isnan(bp))[0] )
angles_nan_fr = np.unique( np.where(np.isnan(angles))[0] )
limbs_nan_fr = np.unique( np.where(np.isnan(limbs))[0] )

In [5]:
angle_power_nan_fr = np.unique( np.where(np.isnan(angle_power))[0] )
limb_power_nan_fr = np.unique( np.where(np.isnan(limb_power))[0] )

In [7]:
from helper import locate_bad_fr

In [8]:
INFO_items = list(INFO.items())
INFO_items.sort(key=lambda x: x[1]['order'])
start_fr = 0   
test = []
for key, file in tqdm(INFO_items):
    save_path = file['directory']
    bp = np.load(f"{PROJECT_PATH}/{save_path}/rotated_bodypoints.npy")
    num_fr, _, _ = bp.shape
    # locate the good frames
    good_fr, bad_fr, disregard_fr = locate_bad_fr(config, bp)
    num_good_fr = len(good_fr)
    start_fr += num_good_fr

In [11]:
start_fr-num_good_fr

563273

In [13]:
tot_good_fr, tot_bad_fr, tot_disregard_fr = locate_bad_fr(config, rot_bp)
len(tot_good_fr)

559988

In [25]:
good_fr_count = 0   
for key, file in tqdm(INFO_items):
    start_fr, stop_fr = file["global_start_fr"], file["global_stop_fr"]
    good_fr, bad_fr, disregard_fr = locate_bad_fr(config, rot_bp[start_fr:stop_fr])
    num_good_fr = len(good_fr)
    good_fr_count += num_good_fr

In [26]:
good_fr_count

563668

In [37]:
tot_good_fr= np.concatenate(tot_good_fr)

In [38]:
print(good_fr_count)
print(tot_good_fr.shape)
print(np.unique(tot_good_fr).shape)

563668
(563668,)
(563668,)


# Analysis

In [ ]:
# Figure Information
max_freq_idx = np.argmax(angle_power[:,:,:], axis=1)
max_freq = freq[max_freq_idx]
num_freq = len(freq)

# Freq Color Scheme
freq_color_palette = sns.color_palette('mako_r', num_freq)
cmap1, norm1 = colors.from_levels_and_colors(freq, freq_color_palette, extend="max")

# Figures
fig, ax = plt.subplots(3,4, figsize=(20,15))
for angle_idx in tqdm(range(12)):
    fig_title = f"{config['angle_labels'][angle_idx]} Frequency"
    freq_cluster_colors = np.array([freq_color_palette[f] for f in max_freq_idx[:,angle_idx]])

    ax[angle_idx%3, angle_idx//3].scatter(all_embed[:,0], all_embed[:,1], color=freq_cluster_colors, alpha=0.005, s=1.5)  
    ax[angle_idx%3, angle_idx//3].set(xlabel='Component 1', ylabel='Component 2', title=fig_title, xlim=region_x_lim, ylim=region_y_lim)

plt.show()

In [ ]:
max_freq, min_freq = config['fps']/2, 1 # Nyquist Frequency
freq = max_freq*2**(-1*np.log2(max_freq/min_freq)*
    (np.arange(config['f_bin'],0,-1)-1)/(config['f_bin']-1))

In [ ]:
# Figure Information
max_freq_idx = np.argmax(angle_power[:,:,:], axis=1)
max_freq = freq[max_freq_idx]
num_freq = len(freq)

# Freq Color Scheme
freq_color_palette = sns.color_palette('mako_r', num_freq)
cmap1, norm1 = colors.from_levels_and_colors(freq, freq_color_palette, extend="max")

# Figures
fig, ax = plt.subplots(3,4, figsize=(28,21))
for angle_idx in tqdm(range(12)):
    fig_title = f"{config['angle_labels'][angle_idx]} Frequency"
    freq_cluster_colors = np.array([freq_color_palette[f] for f in max_freq_idx[:,angle_idx]])

    ax[angle_idx%3, angle_idx//3].scatter(all_embed[:,0], all_embed[:,1], color=freq_cluster_colors, alpha=0.005, s=0.5)  
    ax[angle_idx%3, angle_idx//3].set(xlabel='Component 1', ylabel='Component 2', title=fig_title, xlim=region_x_lim, ylim=region_y_lim)

plt.show()

In [ ]:
# Figure Information
max_freq_idx = np.argmax(limb_power[:,:,:], axis=1)
max_freq = freq[max_freq_idx]
num_freq = len(freq)

# Freq Color Scheme
freq_color_palette = sns.color_palette('mako_r', num_freq)
cmap1, norm1 = colors.from_levels_and_colors(freq, freq_color_palette, extend="max")

# Figures
fig, ax = plt.subplots(3,4, figsize=(28,21))
for angle_idx in tqdm(range(12)):
    fig_title = f"{config['angle_labels'][angle_idx]} Frequency"
    freq_cluster_colors = np.array([freq_color_palette[f] for f in max_freq_idx[:,angle_idx]])

    ax[angle_idx%3, angle_idx//3].scatter(all_embed[:,0], all_embed[:,1], color=freq_cluster_colors, alpha=0.005, s=0.5)  
    ax[angle_idx%3, angle_idx//3].set(xlabel='Component 1', ylabel='Component 2', title=fig_title, xlim=region_x_lim, ylim=region_y_lim)
plt.show()